In [24]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from ST_Encoder import ST_Enc
from args import parse_args
from torch.utils.data import DataLoader, TensorDataset

args = parse_args()
data = np.load('/home/zhangmin/toby/UrbanGPT/data/discharge/data_encoder.npy')
N, T, F = data.shape
print(f"Data shape: N={N}, T={T}, F={F}")

data_mean = np.nanmean(data, axis=(1, 2), keepdims=True)
data_std = np.nanstd(data, axis=(1, 2), keepdims=True)
data_std[data_std == 0] = 1  # 防止除以零
data = (data - data_mean) / data_std

input_window = args.input_window
output_window = args.output_window

data_x = []
data_y = []
mask_y = []

for t in range(T - input_window - output_window + 1):
    x = data[:, t:t+input_window, :].transpose(1, 0, 2)
    y = data[:, t+input_window:t+input_window+output_window, :].transpose(1, 0, 2)
    mask_y.append(~np.isnan(data[:, t+input_window:t+input_window+output_window, 0].transpose(1, 0)))
    data_x.append(x)
    data_y.append(y)

data_x = np.array(data_x)
data_y = np.array(data_y)
mask_y = np.array(mask_y)

print(f"Data_x shape: {data_x.shape}, Data_y shape: {data_y.shape}, Mask_y shape: {mask_y.shape}")

assert data_x.shape[1] == input_window, f"input sequence length not equal to preset sequence length, expected {input_window}, but got {data_x.shape[1]}"

data_x = torch.tensor(data_x, dtype=torch.float32)
data_y = torch.tensor(data_y, dtype=torch.float32)
mask_y = torch.tensor(mask_y, dtype=torch.float32)

eval_dataset = TensorDataset(data_x, data_y, mask_y)
eval_loader = DataLoader(eval_dataset, batch_size=1, shuffle=True)

model = ST_Enc(args, dim_in=F, dim_out=args.output_dim).to('cuda')
checkpoint_path = '/home/zhangmin/toby/UrbanGPT/checkpoints/st_encoder/pretrain_stencoder.pth'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()


def inverse_transform(data, mean, std):
    mean = mean.squeeze()
    std = std.squeeze()
    mean = mean[np.newaxis, np.newaxis, :, np.newaxis]
    std = std[np.newaxis, np.newaxis, :, np.newaxis]
    return data * std + mean


def plot_predictions_vs_true_with_nan(sample_y, predicted_y, mask_y, time_steps, site_index=0):
    plt.figure(figsize=(14, 7))

    true_sediment = sample_y[0, site_index, :, 0]
    predicted_sediment = predicted_y[0, site_index, :, 0]

    true_discharge = sample_y[0, site_index, :, 1]
    predicted_discharge = predicted_y[0, site_index, :, 1]

    nan_mask_sediment = np.array(mask_y[0, site_index, :], dtype=bool)

    plt.scatter(time_steps[~nan_mask_sediment], true_sediment[~nan_mask_sediment], color='k', label='NaN in True Sediment')
    plt.scatter(time_steps[~nan_mask_sediment], predicted_sediment[~nan_mask_sediment], color='m', label='NaN in Predicted Sediment')

    plt.plot(time_steps, true_sediment, label='True Sediment', color='b')
    plt.plot(time_steps, predicted_sediment, label='Predicted Sediment', color='r', linestyle='--')
    plt.plot(time_steps, true_discharge, label='True Discharge', color='g')
    plt.plot(time_steps, predicted_discharge, label='Predicted Discharge', color='orange', linestyle='--')
    plt.xlabel('Time Steps')
    plt.ylabel('Values')
    plt.title(f'Site {site_index} Predictions vs. True Values')
    plt.legend()
    plt.show()


with torch.no_grad():
    for sample_x, sample_y, mask_y in eval_loader:
        sample_x, sample_y, mask_y = sample_x.to('cuda'), sample_y.to('cuda'), mask_y.to('cuda')
        predicted_y, _ = model(sample_x)
        predicted_y = predicted_y.cpu().numpy()
        sample_y = sample_y.cpu().numpy()
        mask_y = mask_y.cpu().numpy()
        predicted_y = inverse_transform(predicted_y, data_mean, data_std)
        sample_y = inverse_transform(sample_y, data_mean, data_std)

        print(predicted_y.shape)
        print(predicted_y[0, :, 0, 0])
        print(sample_y[0, :, 0, 0])

        time_steps = np.arange(len(sample_y[0, :, 0, 0]))  # 时间轴
        plot_predictions_vs_true_with_nan(sample_y, predicted_y, mask_y, time_steps)
        break

Data shape: N=20, T=2811, F=2
Data_x shape: (2788, 12, 20, 2), Data_y shape: (2788, 12, 20, 2), Mask_y shape: (2788, 12, 20)
(1, 12, 20, 2)
[35.75590401 36.87654643 37.53120714 37.66152701 37.46116457 36.88022778
 37.0516238  36.72795542 36.80495381 35.1461558  34.77340918 35.29120108]
[35.00000611 33.99999579 31.99999454 29.99999328 29.00000234 27.00000109
 27.00000109 27.00000109 25.99999077 25.99999077 25.99999077 25.99999077]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 12 but corresponding boolean dimension is 20

<Figure size 1400x700 with 0 Axes>